## Installation

In [ ]:
!pip install -qU bitsandbytes transformers peft accelerate datasets pandas torch

In [2]:
import random
import torch
import pandas as pd
from datasets import Dataset
import peft
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)

set_seed()

## Exploring the data

In [ ]:
df = pd.read_csv("frankenstein_chunks.csv")

In [ ]:
print("Dataframe Info:")
print(df.info())
print("\n")
print("Dataframe Description:")
print(df.describe())
print("\n")
print("Number of unique values in each column:")
print(df.nunique())

print(df.isnull().sum())

In [7]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

## Tokenizing

In [ ]:
model_name = 'mistralai/Mistral-7B-v0.1'

In [ ]:
from huggingface_hub import login

login(token="<HF_API_TOKEN>")

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config)

print(f"Running on: {torch.device("cuda" if torch.cuda.is_available() else "cpu")}")

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=32,
    lora_alpha=64, # This number is usually double the parameter "r".
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

def tokenize_function(examples):
    return tokenizer(examples["text"])

model = get_peft_model(model, config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_train_dataset= train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

## Base Model Performance

In [22]:
def generate_text(prompt):
  device = "cuda"
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=100)
  output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return output

In [ ]:
base_generation = generate_text("I'm afraid I've created a ")
print(base_generation)

In [ ]:
def calc_perplexity(model):
  total_perplexity = 0
  for row in test_dataset:
    inputs = tokenizer(row['text'], return_tensors="pt")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(**inputs, labels=input_ids)

    loss = outputs.loss
    perplexity = torch.exp(loss)
    total_perplexity += perplexity

  num_test_rows = len(test_dataset)
  avg_perplexity = total_perplexity / num_test_rows
  return avg_perplexity

base_ppl = calc_perplexity(model)
print(f"The perplexity score is: {base_ppl}")

## Finetuning

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        warmup_steps=2,
        fp16=True,
        logging_steps=1,
        save_steps=200,
        output_dir="outputs",
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=2e-5,
        optim="paged_adamw_8bit",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

## Final Result

In [ ]:
ft_generation = generate_text("I'm afraid I've created a ")

print("Finetuned generation: " + ft_generation)

## Perplexity Score

In [ ]:
ft_ppl = calc_perplexity(model)
print("Base model perplexity: " + str(base_ppl))
print("Finetuned model perplexity: " + str(ft_ppl))